In [17]:
import os
import pandas as pd

# Replace with file path
directory = '/Users/vincewu/Bees'
log_folder = "Sample"
processed_folder = 'Type 3 (Fully Processed)'
treatment_ids_filename='tagnumeros_09.22.csv'

headers = ["UID", "Day", "Group", "Trip Duration", "Rest Duration",
           "Start Time", "End Time", "Trip Start and End Times"]

activity = [[dict() for days in range(31)] for months in range(12)]
# latest could be December 31
start_month = 11
start_day = 30
# earliest could be January 1
end_month = 0
end_day = 0
# file_count = [[0 for days in range(31)] for months in range(12)]

# for filename in os.listdir(directory+'/'+log_folder):
#     if filename.endswith('.csv'):

#         month = int(filename[file_length-19:file_length-17])-1
#         day = int(filename[file_length-16:file_length-14])-1
#         file_count[month][day]+=1

treatment_ids_csv = pd.read_csv(os.path.join(
    directory, treatment_ids_filename), engine='python', header=None)
treatment_ids = {}

for i in range(len(treatment_ids_csv[0])):
    treatment_ids[treatment_ids_csv.loc[i][1]] = treatment_ids_csv.loc[i][0]

for filename in os.listdir(directory+'/'+log_folder):
    if filename.endswith('.csv'):
        print(filename)
        pathname = os.path.join(directory, log_folder, filename)
        pathname='/Users/vincewu/Bees/Test DPC Data Original/H1R10-H2R11-000008/log_2022_07_18__20_15_17.csv'

        keys = set()
        file_length = len(filename)
        if filename[file_length-6:file_length-5] == ' ':
            subtract = 16
        elif filename[file_length-8:file_length-4] == 'copy':
            subtract=19
        else:
            subtract = 14
        month = int(filename[file_length-(subtract+5):file_length-(subtract+3)])-1
        day = int(filename[file_length-(subtract+2):file_length-subtract])-1

        if month <= start_month:
            if day < start_day:
                start_month = month
                start_day = day
        if month >= end_month:
            if day > end_day:
                end_month = month
                end_day = day

        try:
            df_test = pd.read_csv(pathname, engine='python', sep="[;,. ]", skiprows=23, nrows=1)
            determiner=df_test.columns[0]
        except pd.errors.EmptyDataError:
            determiner='x'

        #DOC
        if determiner == 'Timestamp':
            try:
                df = pd.read_csv(pathname, engine='python',
                                sep="[, ]", skiprows=24, header=None)
            except pd.errors.EmptyDataError:
                df = pd.DataFrame()
            
            if not df.empty:
                df.dropna(axis=1, how='all', inplace=True)

                time_found = False
                temp_columns=[i for i in range (len(df.columns))]
                for i in df.columns:
                    column = str(df.loc[0][i])
                    col_length = len(column)

                    temp_length = len(column)
                    if column[temp_length-6: temp_length-5] == ":" and not time_found:
                        temp_columns[i] = "Timestamp"
                        time_found = True
                    elif column[:4] == "25SQ":
                        temp_columns[i] = "UID"
                    elif column == "1" or column == "2":
                        temp_columns[i] = "Ant. Number"

                df.columns = temp_columns

                for column in df.columns:
                    if column != "Timestamp" and column != "UID" and column != "Ant. Number":
                        del(df[column])

                ids = len(df["UID"])

                for i in range(ids):
                    key = df["UID"][i]
                    if not key in keys:
                        keys.add(key)
                        #print(key)
                        # List with durations of trips in sequential order |
                        # List with durations of time between trips in sequential order |
                        # Start time of first trip | end time of last trip
                        # List with start and end times of all trips
                        activity[month][day][key] = [
                            int(), "x", [], [], int(), int(), []]
                        if key in treatment_ids:
                            activity[month][day][key][1] = treatment_ids[key]
                        else:
                            activity[month][day][key][1] = "No Group"
                        sames = []

                        prev_time = -1
                        add = 0.001
                        for j in range(i, ids):
                            if df["UID"][j] == key:
                                temp = df["Timestamp"][j]
                                time_length = len(temp)
                                # hour = int(temp[:2])
                                # minute = int(temp[3:5])
                                # second = int(float(temp[6:]))
                                hour = int(temp[:time_length-6])
                                minute = int(temp[time_length-5:time_length-3])
                                second = int(float(temp[time_length-2:]))
                                time = hour*3600+minute*60+second
                                if time == prev_time:
                                    time += add
                                    add += 0.001
                                else:
                                    prev_time = time
                                    add = 0.001

                                #print(temp, time, sep=" ")

                                sames.append([time, int(df["Ant. Number"][j])])

                        #print(sames)
                        first_key = True

                        sames.sort()
                        time_groups = []
                        reader_groups = []
                        directions = []

                        index = 0
                        secure_index = -1
                        while index < len(sames):
                            first_time = sames[index][0]
                            time_groups.append([])
                            reader_groups.append([])
                            directions.append("Unknown")
                            time_groups[-1].append(sames[index][0])
                            reader_groups[-1].append(sames[index][1])

                            index += 1
                            while index < len(sames):
                                if sames[index][0]-30 > first_time:
                                    break
                                time_groups[-1].append(sames[index][0])
                                reader_groups[-1].append(sames[index][1])
                                index += 1

                            if len(reader_groups[-1]) >= 2:
                                if reader_groups[-1][0] == 1 and reader_groups[-1][-1] == 2:
                                    directions[-1] = "Arriving"
                                elif reader_groups[-1][0] == 2 and reader_groups[-1][-1] == 1:
                                    directions[-1] = "Departing"

                                if secure_index == -1 and (directions[-1] == "Arriving" or directions[-1] == "Departing"):
                                    secure_index = len(directions)-1

                        time_groups = [[int(i) for i in block]
                                    for block in time_groups]
                        #found a secure arrival or departure
                        if i != -1:
                            possible_directions = [directions[secure_index], "x"]
                            if directions[secure_index] == "Arriving":
                                possible_directions[1] = "Departing"
                            else:
                                possible_directions[1] = "Arriving"

                            for i in range(len(directions)):
                                if directions[i] == "Unknown":
                                    distance = abs(i-secure_index) % 2
                                    directions[i] = possible_directions[distance]

                        #if no secure arrival or departure, everything is unknown
                        #didn't happen a single time in the sample

                        prev_time = -1
                        index = 0
                        while index < len(time_groups)-1 and directions[index] == "Arriving":
                            index += 1
                        # latest "Departing", earliest "Arriving"
                        while index < len(time_groups):
                            while index < len(time_groups)-1 and directions[index+1] == "Departing":
                                index += 1
                            # ends on a series of "Departing"
                            if index == len(time_groups)-1:
                                break

                            start_time = time_groups[index][0]
                            # first trip of the day for the bee
                            if prev_time == -1:
                                activity[month][day][key][4] = start_time
                            else:
                                activity[month][day][key][3].append(
                                    start_time-prev_time)
                            activity[month][day][key][6].append([start_time])

                            index += 1
                            end_time = time_groups[index][0]
                            activity[month][day][key][2].append(
                                end_time-start_time)
                            activity[month][day][key][5] = end_time
                            activity[month][day][key][6][-1].append(end_time)
                            prev_time = end_time

                            while index < len(time_groups)-1 and directions[index+1] == "Arriving":
                                index += 1

                # for key in activity[month][day]:
                #     print(key, ": ", activity[month][day][key])

                # if file_count[month][day]==0:

                month_df = pd.DataFrame.from_dict(activity[month][day], orient="index")
                month_df.to_csv(directory+'/'+processed_folder+'/type3_' +
                                str(month+1)+'_'+str(day+1)+".csv", header=False)
        # DPC (Arriving, Departing)
        else:
            try:
                df = pd.read_csv(pathname, engine='python',
                                 sep="[.,; ]", skiprows=24, header=None)
            except pd.errors.EmptyDataError:
                df = pd.DataFrame()

            if not df.empty:
                df.dropna(axis=1, how='all', inplace=True)

                time_found = False
                temp_columns = [i for i in range(len(df.columns))]
                for i in df.columns:
                    column = str(df.loc[0][i])
                    col_length = len(column)

                    temp_length = len(column)
                    if column[temp_length-6: temp_length-5] == ":" and not time_found:
                        temp_columns[i] = "Timestamp"
                        time_found = True
                    elif column[:4] == "25SQ":
                        temp_columns[i] = "UID"
                    elif column == "Arriving" or column == "Departing" or column == "Unknown":
                        temp_columns[i] = "Direction"

                df.columns = temp_columns

                for column in df.columns:
                    if column != "Timestamp" and column != "UID" and column != "Direction":
                        del(df[column])

                print
                
                ids = len(df["UID"])

                for i in range(ids):
                    key = df["UID"][i]
                    if not key in keys:
                        keys.add(key)
                        #print(key)
                        # List with durations of trips in sequential order |
                        # List with durations of time between trips in sequential order |
                        # Start time of first trip | end time of last trip
                        # List with start and end times of all trips
                        activity[month][day][key] = [
                            int(), "x", [], [], int(), int(), []]
                        if key in treatment_ids:
                            activity[month][day][key][1] = treatment_ids[key]
                        else:
                            activity[month][day][key][1] = "No Group"
                        sames = []

                        prev_time = -1
                        add = 0.001
                        for j in range(i, ids):
                            if df["UID"][j] == key:
                                temp = df["Timestamp"][j]
                                time_length = len(temp)
                                # hour = int(temp[:2])
                                # minute = int(temp[3:5])
                                # second = int(float(temp[6:]))
                                hour = int(temp[:time_length-6])
                                minute = int(temp[time_length-5:time_length-3])
                                second = int(float(temp[time_length-2:]))
                                time = hour*3600+minute*60+second
                                if time == prev_time:
                                    time += add
                                    add += 0.001
                                else:
                                    prev_time = time
                                    add = 0.001

                                #print(temp, time, sep=" ")

                                sames.append([time, df['Direction'][j]])

                        #print(sames)
                        first_key = True

                        #sames.sort()
                        print(sames)
#                         time_groups = []
#                         # reader_groups = []
#                         directions = []

#                         index = 0
#                         secure_index = -1
#                         while index < len(sames):
#                             first_time = sames[index][0]
#                             time_groups.append([])
#                             # reader_groups.append([])
#                             # directions.append("Unknown")
#                             time_groups[-1].append(sames[index][0])
#                             directions[-1].append(sames[index][1])

#                             index += 1
#                             while index < len(sames):
#                                 if sames[index][0]-30 > first_time:
#                                     break
#                                 time_groups[-1].append(sames[index][0])
#                                 directions[-1].append(sames[index][1])
#                                 index += 1

#                             if len(reader_groups[-1]) >= 2:
#                                 if reader_groups[-1][0] == 1 and reader_groups[-1][-1] == 2:
#                                     directions[-1] = "Arriving"
#                                 elif reader_groups[-1][0] == 2 and reader_groups[-1][-1] == 1:
#                                     directions[-1] = "Departing"

#                                 if secure_index == -1 and (directions[-1] == "Arriving" or directions[-1] == "Departing"):
#                                     secure_index = len(directions)-1

#                         time_groups = [[int(i) for i in block]
#                                        for block in time_groups]
#                         #found a secure arrival or departure
#                         if i != -1:
#                             possible_directions = [
#                                 directions[secure_index], "x"]
#                             if directions[secure_index] == "Arriving":
#                                 possible_directions[1] = "Departing"
#                             else:
#                                 possible_directions[1] = "Arriving"

#                             for i in range(len(directions)):
#                                 if directions[i] == "Unknown":
#                                     distance = abs(i-secure_index) % 2
#                                     directions[i] = possible_directions[distance]

#                         #if no secure arrival or departure, everything is unknown
#                         #didn't happen a single time in the sample

#                         prev_time = -1
#                         index = 0
#                         while index < len(time_groups)-1 and directions[index] == "Arriving":
#                             index += 1
#                         # latest "Departing", earliest "Arriving"
#                         while index < len(time_groups):
#                             while index < len(time_groups)-1 and directions[index+1] == "Departing":
#                                 index += 1
#                             # ends on a series of "Departing"
#                             if index == len(time_groups)-1:
#                                 break

#                             start_time = time_groups[index][0]
#                             # first trip of the day for the bee
#                             if prev_time == -1:
#                                 activity[month][day][key][4] = start_time
#                             else:
#                                 activity[month][day][key][3].append(
#                                     start_time-prev_time)
#                             activity[month][day][key][6].append([start_time])

#                             index += 1
#                             end_time = time_groups[index][0]
#                             activity[month][day][key][2].append(
#                                 end_time-start_time)
#                             activity[month][day][key][5] = end_time
#                             activity[month][day][key][6][-1].append(end_time)
#                             prev_time = end_time

#                             while index < len(time_groups)-1 and directions[index+1] == "Arriving":
#                                 index += 1

#                 # for key in activity[month][day]:
#                 #     print(key, ": ", activity[month][day][key])

#                 # if file_count[month][day]==0:

#                 month_df
#                 month_df = pd.DataFrame.from_dict(
#                     activity[month][day], orient="index")
#                 month_df.to_csv(directory+'/'+processed_folder+'/type3_' +
#                                 str(month+1)+'_'+str(day+1)+".csv", header=False)

# # months=["January","February","March","April","May","June","July",
# #             "August","September","October","November","December"]
# months_num = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
# # for month in range (len(activity)):
# #     print(months[month])
# #     for day in range (months_num[month]):
# #         print("Day: ", day)
# #         print(activity[month][day])

# # final_df=[]
# # day=1
# # for i in range(start_day, months_num[start_month]):
# #     for key in activity[start_month][i]:
# #         activity[start_month][i][key][0]=day

# #     final_df.append(activity[start_month][i])
# #     day+=1

# # for i in range(start_month+1, end_month+1):
# #     for j in range(months_num[i]):
# #         for key in activity[i][j]:
# #             activity[i][j][key][0]=day

# #         final_df.append(activity[i][j])
# #         day+=1

# # for i in range(end_day+1):
# #     for key in activity[end_month][i]:
# #         activity[end_month][i][key][0]=day

# #     final_df.append(activity[end_month][i])
# #     day+=1

# # final_df

# #headers=[]
# # print(start_month, start_day, end_month, end_day, day, sep=" ")

# # final_df=pd.DataFrame.to_csv(final_df)

# # final_df.to_csv(directory+'/'+processed_folder+'/processed.csv', header=False)

# # List with durations of trips in sequential order |
# # List with durations of time between trips in sequential order |
# # Start time of first trip | end time of last trip
# # List with start and end times of all trips


log_2022_09_26__xx_xx_xx.csv
   Timestamp                    UID Direction
0   13:15:16  25SQCMICSBBB67800AA89       NaN
1   13:15:26  25SQCMICSBBB67800AA89       NaN
2   13:15:26  25SQCMICSBB5A7300AFFC         x
3   13:15:36  25SQCMICSBB5A7300AFFC         x
4   13:15:36  25SQCMICSBBB67800AA89       NaN
5   13:15:46  25SQCMICSBBB67800AA89       NaN
6   13:15:46  25SQCMICSBB3E5C00B009         x
7   13:15:47  25SQCMICSBB5CA800B097       NaN
8   13:15:47  25SQCMICSBB9FBE00B07A         x
9   13:15:56  25SQCMICSBBB67800AA89       NaN
10  13:15:56  25SQCMICSBB455F00B089         x
11  13:15:56  25SQCMICSBBF23800AA18         x
12  13:15:56  25SQCMICSBBE30F00B03A         x
13  13:15:57  25SQCMICSBB9FBE00B07A         x
14  13:16:06  25SQCMICSBBB67800AA89       NaN
15  13:16:07  25SQCMICSBB5CA800B097       NaN
16  13:16:16  25SQCMICSBBB67800AA89       NaN
17  13:16:17  25SQCMICSBB5CA800B097         x
18  13:16:17  25SQCMICSBB48A200AA01         x
19  13:16:36  25SQCMICSBB0CE200AA90         x
20  1

In [11]:
pathname='/Users/vincewu/Bees/Test DPC Data Original/H1R10-H2R11-000008/log_2022_07_18__20_15_17.csv'
df = pd.read_csv(pathname, engine='python',
                                 sep="[;., ]", skiprows=24, header=None)
df.dropna(axis=1, how='all', inplace=True)
for i in df.columns:
    print(i)

0
1
2
3
4
5
7
8
9
10
11
12
13
14
15
16
17
